# CA2 Rory Sheridan

# Question 1
Read in the below information of a company’s transactions for the year2022 into a python dictionary files.

### a. Convert each database into a dataframe in python.

In [15]:
import pandas as pd

In [16]:
# Declare data.
dbase1_dict = {
    'TR_ID': [
        'C23',
        'C24',
        'C25',
        'C26',
        'C27',
        'C28',
    ],
    'Product': [
        'Telephone',
        'Monitors',
        'Car',
        'Rent',
        'Services',
        'Training',
    ],
    'Quantitys': [
        420,
        'Ten',
        234,
        114,
        56,
        13,
    ],
    'Purchase_prices': [
        223,
        344,
        223,
        'Tenn',
        'Two',
        223,
    ],
}
dbase2_dict = {
    'TR_ID': [
        'C23',
        'C24',
        'C25',
        'C26',
        'C27',
        'C28',
    ],
    'Products': [
        'Telephone',
        'Monitors',
        'Car',
        'Rent',
        'Services',
        'Training',
    ],
    'Sale_price': [
        300,
        435,
        329,
        544,
        'thirty',
        329,
    ],
    'Debit': [
        2,
        2,
        2,
        2,
        2,
        2,
    ],
}

dbase3_dict = {
    'TR_ID': ['C28'],
    'Product': ['Training'],
    'Quantity': [13],
    'Purchase_price': [223],
    'Sale_price': [329],
}

In [17]:
dbase1 = pd.DataFrame(dbase1_dict)
dbase1

,TR_ID,Product,Quantitys,Purchase_prices
0,C23,Telephone,420,223
1,C24,Monitors,Ten,344
2,C25,Car,234,223
3,C26,Rent,114,Tenn
4,C27,Services,56,Two
5,C28,Training,13,223


In [18]:
dbase2 = pd.DataFrame(dbase2_dict)
dbase2

,TR_ID,Products,Sale_price,Debit
0,C23,Telephone,300,2
1,C24,Monitors,435,2
2,C25,Car,329,2
3,C26,Rent,544,2
4,C27,Services,thirty,2
5,C28,Training,329,2


In [19]:
dbase3 = pd.DataFrame(dbase3_dict)
dbase3

,TR_ID,Product,Quantity,Purchase_price,Sale_price
0,C28,Training,13,223,329
